<a href="https://colab.research.google.com/github/SyncHJ/ESG_MBA_data_analytics/blob/main/ESG_MBA_10%EC%9B%94_08%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 453 kB 62.4 MB/s 


In [ ]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [ ]:
import re
import os
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import nltk
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc

import xlsxwriter
from openpyxl import Workbook

font_path ='./NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
rc('font', family='AppleGothic')

okt = Okt()

# driver_path = r'/Users/mac_h/Desktop'
# os.chdir(driver_path)
data = pd.read_excel('/content/ESG_sp.xlsx',engine='openpyxl')
# csa=pd.read_csv("ESG_sp.csv", encoding='cp949')

In [ ]:
print(len(data)) #데이터 값이 몇 개 있는지 확인

10438


In [ ]:
print(data.isnull().values.any()) #결측치 값 유무 확인 (true -> 결측치 있음, False -> 결측치 없음)

True


In [ ]:
import re
 
def preprocessing(text):
    # 특수문자나 이모티콘 등 아래의 특수기호들을 제거합니다(%등은 남김).
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   
    #  re.sub는 위에서 특수문자를 제거한 text에서 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

def clean_text(text):
    text = str(text)
    text = re.sub('\S*@\S*\s?', '', text) # 기자 emails 제거
    text = re.sub(r'\[.*?\]', '', text) # 대괄호 안에 텍스트 제거 : 뉴스이름 + 기자이름
    text = re.sub(r'\(.*?\)', '', text) # 소괄호 안에 텍스트 제거 : 뉴스이름 + 기자이름
    text = re.sub(r'\w*\d\w*', '', text) # 숫자 포함하는 텍스트 제거

    text = re.sub('http[s]?://\S+', '', text) # http url 제거
    
#     for p in press:
#         text = re.sub(str(p),'',text)
#     # 특수문자나 이모티콘 등 아래의 특수기호들을 제거합니다(%등은 남김).
#     text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    
    # 위에서 특수문자를 제거한 text에서 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    
    # text = re.sub('\s* 기자', "", text)
    text = re.sub("\n", '', text) #개행문자 제거
    text = re.sub("\xa0", '', text) #개행문자 제거

    return text

In [ ]:
# 불용어 제거 작업
import json #json 파일을 불러오기 한 뒤에 아래 Stopworkd.extend에서 추가를 해서 지울 것들 만들면 됨
with open('korean_stopword.json',encoding='utf8') as f:  # 분석하고자 파일과 동일한 폴더(위치)에 있어야 한다
    stopwords = json.load(f)

In [ ]:
cs = pd.DataFrame()
cs['keywords'] = data['keywords'] # content와 sentiment 계산 # column이 하나일 때는 [] 를 한번만 사용


cs['text_clean']=pd.DataFrame(cs['keywords'].apply(lambda x: clean_text(x)))

# stopwords를 하기 위해서는 tokenize를 먼저 해야 함 (계속해서 불필요한 단어들을 보면서 제거)
cs['text_ko'] = [okt.morphs(doc) for doc in cs.text_clean]
cs['text_tokenized'] = cs.text_ko.apply(lambda x: [ a for a in x if a not in stopwords])

In [ ]:
print(cs)

# text_clean 된 것만 사용해서 분석은 안되나요?????????????
# tokenized되니 단어가 너무 쪼개지는 것 같아서....

                                                keywords  \
0      SK케미칼,발간,TCFD,보고서,기후변화,공유,대응,전략,SK케미칼,생산,제품,생애...   
1      제주은행,사회,공헌비율,최고,은행,제주은행,시중,은행,이익,대비,사회,공헌,비율,국...   
2      개최,KICOX,국민,참여,혁신,아이디어,공모전,한국산업단지공단,국민,목소리,아이디...   
3      SK케미칼,기후,변화,대응,전략,TCFD,보고서,발간,SK,케미칼,TCFD,보고서,...   
4      삼성,LG전자,KES,참가,혁신제품,경쟁,삼성전자,스마트싱스,활용,체험존,마련,삼성...   
...                                                  ...   
10433  예산,심덕섭,고창,군수,취임,고창군,확보,취임식,민선,심덕섭,전북,고창,군수,고창,...   
10434  인천도시공사,수상,ESG,공공,기관,종합,대상,지방공기업,발행,ESG,채권,ESG,...   
10435  예방,대청호,녹조,처리,금강,유역,홍수,가뭄,예방,재해,주목적,준공,대청호,대전,충...   
10436  순천향대,지주,자회사,IR,대회,Value-UP,R&BD,세미나,개최,아산,순천향대...   
10437  국립,생태원,한국가스공사,업무협약,멸종,위기,보전,업무,협약,서천,국립생태원,원장,...   

                                              text_clean  \
0      SK케미칼 발간 TCFD 보고서 기후변화 공유 대응 전략 SK케미칼 생산 제품 생애...   
1      제주은행 사회 공헌비율 최고 은행 제주은행 시중 은행 이익 대비 사회 공헌 비율 국...   
2      개최 KICOX 국민 참여 혁신 아이디어 공모전 한국산업단지공단 국민 목소리 아이디...   
3      SK케미칼 기후 변화 대응 전략 TCFD 보고서 발간 SK

In [ ]:
tokenized_doc=cs['text_tokenized']

## Word2Vec

In [ ]:
tokenized_do= list(tokenized_doc)

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

tokenizer.fit_on_texts(tokenized_do)

In [ ]:
# 정수 인코딩
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_do)

vocab_size = len(word2idx) + 1 
print('단어 집합의 크기 :', vocab_size) #단어 집합의 크기 : 812

단어 집합의 크기 : 37932


In [ ]:
import gensim
from gensim.models import Word2Vec, FastText

model = Word2Vec(tokenized_do,  min_count=3)

In [ ]:
model.wv.vectors.shape #(189, 100)

(21848, 100)

In [ ]:
from gensim.models import KeyedVectors
import os

model.wv.save_word2vec_format('jang_ta') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('jang_ta')
!python -m gensim.scripts.word2vec2tensor --input jang_ta --output jang_ta

2022-10-05 05:37:51,072 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py --input jang_ta --output jang_ta
2022-10-05 05:37:51,072 - utils_any2vec - INFO - loading projection weights from jang_ta
2022-10-05 05:37:52,304 - utils_any2vec - INFO - loaded (21848, 100) matrix from jang_ta
2022-10-05 05:37:54,046 - word2vec2tensor - INFO - 2D tensor file saved to jang_ta_tensor.tsv
2022-10-05 05:37:54,046 - word2vec2tensor - INFO - Tensor metadata file saved to jang_ta_metadata.tsv
2022-10-05 05:37:54,049 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [ ]:
# 가시화 하는 방법은... 카톡으로 드린 이미지 참고!

## K-means

In [ ]:
cs['text_tokenized_str']=cs['text_tokenized'].apply(lambda x: " ".join(x))
cs['text_tokenized_str']

0        SK 케미칼 발간 TCFD 보고서 기후변화 공유 대응 전략 SK 케미칼 생산 제품 ...
1        제주 은행 사회 공헌 비율 최고 은행 제주 은행 시중 은행 이익 대비 사회 공헌 비...
2        개최 KICOX 국민 참여 혁신 아이디어 공모전 한국 산업 공단 국민 목소리 아이디...
3        SK 케미칼 기후 변화 대응 전략 TCFD 보고서 발간 SK 케미칼 TCFD 보고서...
4        삼성 LG KES 참가 혁신 제품 경쟁 삼성 스마트 싱스 활용 체험 존 마련 삼성 ...
                               ...                        
10433    예산 심덕섭 고창 군수 취임 고창군 확보 취임식 민선 심덕섭 전북 고창 군수 고창 ...
10434    인천 도 시공사 수상 ESG 공공 기관 종합 대상 지방 공기업 발행 ESG 채권 E...
10435    예방 대청호 녹조 처리 금강 유역 홍수 가뭄 예방 재해 주 목적 준공 대청호 대전 ...
10436    순천향대 지주 자회사 IR 대회 Value UP R BD 세미나 개최 아산 순천향대...
10437    국립 생태 원 한국 가스 공사 업무 협약 멸종 위기 보전 업무 협약 서천 국립생태원...
Name: text_tokenized_str, Length: 10438, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer(stop_words='english',ngram_range=(1,2), min_df=0.05,max_df=0.95)
feature_vect=tfidfv.fit_transform(cs['text_tokenized_str'])

In [ ]:
from sklearn.cluster import KMeans

# 10개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=20, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [ ]:
print(cluster_label)
print(cluster_centers)

[ 5 15 10 ...  0 10 14]
[[0.00264027 0.02466952 0.01723571 ... 0.00845778 0.0009702  0.00143949]
 [0.00156828 0.04038254 0.01446241 ... 0.0009911  0.0390046  0.02302794]
 [0.01307438 0.02134682 0.01208433 ... 0.05120065 0.00105331 0.00130013]
 ...
 [0.01325755 0.02488062 0.01319412 ... 0.00468638 0.00845469 0.00539309]
 [0.00088099 0.03821905 0.01223898 ... 0.014762   0.00016445 0.00183061]
 [0.00307286 0.02440744 0.00936643 ... 0.00423582 0.01676136 0.01337775]]


In [ ]:
cs['cluster_label'] = cluster_label
cs.head()

,keywords,text_clean,text_ko,text_tokenized,text_tokenized_str,cluster_label
0,"SK케미칼,발간,TCFD,보고서,기후변화,공유,대응,전략,SK케미칼,생산,제품,생애...",SK케미칼 발간 TCFD 보고서 기후변화 공유 대응 전략 SK케미칼 생산 제품 생애...,"[SK, 케미칼, 발간, TCFD, 보고서, 기후변화, 공유, 대응, 전략, SK,...","[SK, 케미칼, 발간, TCFD, 보고서, 기후변화, 공유, 대응, 전략, SK,...",SK 케미칼 발간 TCFD 보고서 기후변화 공유 대응 전략 SK 케미칼 생산 제품 ...,5
1,"제주은행,사회,공헌비율,최고,은행,제주은행,시중,은행,이익,대비,사회,공헌,비율,국...",제주은행 사회 공헌비율 최고 은행 제주은행 시중 은행 이익 대비 사회 공헌 비율 국...,"[제주, 은행, 사회, 공헌, 비율, 최고, 은행, 제주, 은행, 시중, 은행, 이...","[제주, 은행, 사회, 공헌, 비율, 최고, 은행, 제주, 은행, 시중, 은행, 이...",제주 은행 사회 공헌 비율 최고 은행 제주 은행 시중 은행 이익 대비 사회 공헌 비...,15
2,"개최,KICOX,국민,참여,혁신,아이디어,공모전,한국산업단지공단,국민,목소리,아이디...",개최 KICOX 국민 참여 혁신 아이디어 공모전 한국산업단지공단 국민 목소리 아이디...,"[개최, KICOX, 국민, 참여, 혁신, 아이디어, 공모전, 한국, 산업, 단지,...","[개최, KICOX, 국민, 참여, 혁신, 아이디어, 공모전, 한국, 산업, 공단,...",개최 KICOX 국민 참여 혁신 아이디어 공모전 한국 산업 공단 국민 목소리 아이디...,10
3,"SK케미칼,기후,변화,대응,전략,TCFD,보고서,발간,SK,케미칼,TCFD,보고서,...",SK케미칼 기후 변화 대응 전략 TCFD 보고서 발간 SK 케미칼 TCFD 보고서 ...,"[SK, 케미칼, 기후, 변화, 대응, 전략, TCFD, 보고서, 발간, SK, 케...","[SK, 케미칼, 기후, 변화, 대응, 전략, TCFD, 보고서, 발간, SK, 케...",SK 케미칼 기후 변화 대응 전략 TCFD 보고서 발간 SK 케미칼 TCFD 보고서...,5
4,"삼성,LG전자,KES,참가,혁신제품,경쟁,삼성전자,스마트싱스,활용,체험존,마련,삼성...",삼성 LG전자 KES 참가 혁신제품 경쟁 삼성전자 스마트싱스 활용 체험존 마련 삼성...,"[삼성, LG, 전자, KES, 참가, 혁신, 제품, 경쟁, 삼성, 전자, 스마트,...","[삼성, LG, KES, 참가, 혁신, 제품, 경쟁, 삼성, 스마트, 싱스, 활용,...",삼성 LG KES 참가 혁신 제품 경쟁 삼성 스마트 싱스 활용 체험 존 마련 삼성 ...,12


In [ ]:
cs[cs['cluster_label']==1].sort_values(by='text_tokenized_str')

,keywords,text_clean,text_ko,text_tokenized,text_tokenized_str,cluster_label
7447,"AGC,테크노한국,양포도서관,10대,E-book,리더기,전자책,기증,AGC화인테크노...",AGC 테크노한국 양포도서관 E book 리더기 전자책 기증 AGC화인테크노한국 ...,"[AGC, 테크노, 한국, 양, 포, 도서관, E, book, 리더, 기, 전자책,...","[AGC, 테크노, 한국, 양, 포, 도서관, E, book, 리더, 기, 전자책,...",AGC 테크노 한국 양 포 도서관 E book 리더 기 전자책 기증 AGC 화인 테...,1
7393,"AGC화인테크노한국,양포도서관,경북,구미,양포,도서관,리더기,전자책,기증,ESG,경...",AGC화인테크노한국 양포도서관 경북 구미 양포 도서관 리더기 전자책 기증 ESG 경...,"[AGC, 화인, 테크노, 한국, 양, 포, 도서관, 경북, 구미, 양, 포, 도서...","[AGC, 화인, 테크노, 한국, 양, 포, 도서관, 경북, 구미, 양, 포, 도서...",AGC 화인 테크노 한국 양 포 도서관 경북 구미 양 포 도서관 리더 기 전자책 기...,1
4170,"BBQ,아이러브,아프리카,영향력,확산,사회적,앞장,가치,실현,최대,프랜차이즈,제너시...",BBQ 아이러브 아프리카 영향력 확산 사회적 앞장 가치 실현 최대 프랜차이즈 제너시...,"[BBQ, 아이, 러브, 아프리카, 영향력, 확산, 사회, 적, 앞장, 가치, 실현...","[BBQ, 러브, 아프리카, 영향력, 확산, 사회, 적, 앞장, 가치, 실현, 최대...",BBQ 러브 아프리카 영향력 확산 사회 적 앞장 가치 실현 최대 프랜차이즈 제너시스...,1
6145,"BC카드,기부,특화,밥바라밥,페이,북머니,체크,카드,출시,이용액,0.1%,청년,취약...",BC카드 기부 특화 밥바라밥 페이 북머니 체크 카드 출시 이용액 청년 취약 계...,"[BC, 카드, 기부, 특화, 밥, 바라, 밥, 페이, 북, 머니, 체크, 카드, ...","[BC, 카드, 기부, 특화, 밥, 바라, 밥, 페이, 북, 머니, 체크, 카드, ...",BC 카드 기부 특화 밥 바라 밥 페이 북 머니 체크 카드 출시 용액 청년 취약 계...,1
3701,"BNK신용정보,추석,이웃,온정,ESG,실천,대표이사,BNK,신용,정보,김성주,대표,...",BNK신용정보 추석 이웃 온정 ESG 실천 대표이사 BNK 신용 정보 김성주 대표 ...,"[BNK, 신용정보, 추석, 이웃, 온정, ESG, 실천, 대표이사, BNK, 신용...","[BNK, 신용정보, 추석, 이웃, 온정, ESG, 실천, 대표이사, BNK, 신용...",BNK 신용정보 추석 이웃 온정 ESG 실천 대표이사 BNK 신용 정보 김성주 대표...,1
...,...,...,...,...,...,...
4705,"효성첨단소재,효성화학,친환경,금상첨화,챌린지,효성첨단소재,효성화학,신기업가정신,실천...",효성첨단소재 효성화학 친환경 금상첨화 챌린지 효성첨단소재 효성화학 신기업가정신 실천...,"[효성, 첨단, 소재, 효성, 화학, 친환경, 금상첨화, 챌, 린지, 효성, 첨단,...","[효성, 첨단, 소재, 효성, 화학, 친환경, 금상첨화, 챌, 린지, 효성, 첨단,...",효성 첨단 소재 효성 화학 친환경 금상첨화 챌 린지 효성 첨단 소재 효성 화학 신 ...,1
515,"효성,호국,보답,애국,국가유공자,사회적,앞장,효성그룹,슬로건,애국지사,호국,보훈,활...",효성 호국 보답 애국 국가유공자 사회적 앞장 효성그룹 슬로건 애국지사 호국 보훈 활...,"[효성, 호국, 보답, 애국, 국가, 유공, 자, 사회, 적, 앞장, 효성, 그룹,...","[효성, 호국, 보답, 애국, 국가, 유공, 사회, 적, 앞장, 효성, 그룹, 슬로...",효성 호국 보답 애국 국가 유공 사회 적 앞장 효성 그룹 슬로건 애국지사 호국 보훈...,1
7866,"휠체어,재활치료,활발,장애,아동,지원,경영대상,K-ESG,경영,대상,주식회사,상상,...",휠체어 재활치료 활발 장애 아동 지원 경영대상 K ESG 경영 대상 주식회사 상상 ...,"[휠체어, 재활, 치료, 활발, 장애, 아동, 지원, 경영, 대상, K, ESG, ...","[휠체어, 재활, 치료, 활발, 장애, 아동, 지원, 경영, 대상, K, ESG, ...",휠체어 재활 치료 활발 장애 아동 지원 경영 대상 K ESG 경영 대상 주식회사 상...,1
8128,"휴비스,전주,공장,희망키움,도서,물품,전달,임직원,휴비스,전주,공장,남관초등학교,완...",휴비스 전주 공장 희망키움 도서 물품 전달 임직원 휴비스 전주 공장 남관초등학교 완...,"[휴비스, 전주, 공장, 희망, 키움, 도서, 물품, 전달, 임, 직원, 휴비스, ...","[휴비스, 전주, 공장, 희망, 키움, 도서, 물품, 전달, 임, 직원, 휴비스, ...",휴비스 전주 공장 희망 키움 도서 물품 전달 임 직원 휴비스 전주 공장 남관 초등학...,1


In [ ]:
# 군집별 핵심 단어 추출하기
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (20, 459)
[[0.00264027 0.02466952 0.01723571 ... 0.00845778 0.0009702  0.00143949]
 [0.00156828 0.04038254 0.01446241 ... 0.0009911  0.0390046  0.02302794]
 [0.01307438 0.02134682 0.01208433 ... 0.05120065 0.00105331 0.00130013]
 ...
 [0.01325755 0.02488062 0.01319412 ... 0.00468638 0.00845469 0.00539309]
 [0.00088099 0.03821905 0.01223898 ... 0.014762   0.00016445 0.00183061]
 [0.00307286 0.02440744 0.00936643 ... 0.00423582 0.01676136 0.01337775]]


In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        
    return cluster_details

In [ ]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('='*120)

In [ ]:
feature_names = tfidfv.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=cs,
                                  feature_names=feature_names, clusters_num=20, top_n_features=30 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['제품', '플라스틱', '사용', '생산', '고객', '상품', '소재', '판매', '원료', '브랜드', '환경', '출시', '매출', '바이오', '재활용', '시장', '기업', '소비자', '공장', '적용', '서비스', '소비', '가격', '가능', '경영', '대표', '확대', '활용', '인증', '제공']
####### Cluster 1
Top features: ['기부', '캠페인', '사회', '지역', '활동', '직원', '전달', '공헌', '사회 공헌', '지원', '실천', '진행', '공헌 활동', '계층', '지역 사회', '참여', '환경', '경영', '취약', '취약 계층', '그룹', '행사', '증권', '복지', 'esg 경영', '기업', '후원', '한국', '사용', '건강']
####### Cluster 2
Top features: ['에너지', '재생 에너지', '재생', '전력', '탄소', '기업', '사업', '발전', '생산', '사용', '효율', '중립', '설비', '탄소 중립', '배출', '산업', '전환', '가입', '환경', '공급', '공장', '가스', '투자', '한국', '글로벌', 'lg', '전기', '감축', '정부', '솔루션']
####### Cluster 3
Top features: ['기업', '경영', 'esg 경영', '대표', '사회', '회장', '평가', '위원회', '한국', '환경', '공시', '지속', '그룹', '가능', '경제', '지속 가능', '글로벌', '사업', '성장', '전략', '이사', '가치', '산업', '지원', '기업 esg', '투자', '대응', '책임', '회사', '강화']
####### Cluster 4
Top features: ['탄소', '중립', '탄소 중립', '배출', '감축', '가스', '온실 가스', '온실', '탄소 배출', '기업',

## LDA

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(cs['text_tokenized'])
print(dictionary)

NameError: ignored

In [ ]:
corpus = [dictionary.doc2bow(text) for text in cs['text_tokenized']]

print(corpus[0])

for i in range(5):
    print(dictionary[i])

In [ ]:
tokenized = cs['text_tokenized']

In [ ]:
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
perplexity_values =[]
coherence_values=[]
for i in range(5,30):
    ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)
    coherence_model_lda =CoherenceModel(model=ldamodel, texts=tokenized, dictionary = dictionary,topn=10)
    coherence_lda=coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

    perplexity_values.append(ldamodel.log_perplexity(corpus))

In [ ]:
x=range(5,30)
plt.plot(x,coherence_values)
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show() #number of topics= 18

In [ ]:
plt.plot(x,perplexity_values)
plt.xlabel('number of topics')
plt.ylabel('perplexity score')
plt.show() 

In [ ]:
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=18, 
                                         alpha=0.1, id2word=dictionary)

In [ ]:
ldamodel.print_topics(num_words=5)

In [ ]:
!pip install pyLDAvis

In [ ]:
#시각화
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)